In [1]:
import json
import numpy as np
import pandas as pd
import lxml
import re
import os
from datetime import datetime, date, time
#import pyodbc
import requests
from tqdm import tqdm

### Get EA script history in JSON format via the API

In [2]:
extract_date = '2023-04-05'

In [3]:
custom_headers  = {"Authorization": "Gt4qkl1ahFDcFyQwfjMBYT0utbLD4STHQjWdIXzsYH8v3w29M6Tm6tVKI0uMj6Qh", "Content-Type": "application/json"}
url = 'https://ea.executive.automats.app/api/bi/scripts?withHistory=true'
#url = 'https://ea.executive.automats.app/api/bi/scripts?withHistory=true&filter[where][startTime][gt]=2023-03-20'
# url = 'https://ea.executive.automats.app/api/bi/scripts?withHistory=true&filter[where][createdAt][gt]=2023-02-27'

resp = requests.get(url, headers = custom_headers, verify=False)
print(resp.status_code)

# Previous runtime = 2min 55s

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:999: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ea.executive.automats.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


200


In [4]:
json_data = json.loads(resp.text)
json_text = json.dumps(json_data, indent=4)

# Previous runtime = 2min 2s

In [5]:
try:
    os.remove('data/execution/try.json')
except:
    print('File does not exist')

with open('data/execution/try.json', 'w') as fw:
    fw.write(json_text)

### Remove invalid characters from the inbound JSON data

In [6]:
try:
    os.remove('data/execution/script_history_fixed.json')
except:
    print('File does not exist')

#with open('data/execution/script_history.json', 'r') as fr:
with open('data/execution/try.json', 'r') as fr:    
    # reading line by line
    lines = fr.readlines()
    last_line = len(lines)
    print(len(lines))
    
    for line in tqdm(lines):
        line = re.sub("\u0003", "", line)
        with open('data/execution/script_history_fixed.json', 'a') as fw:
            fw.write(line) 

# Previous runtime = 5m 28s

25448995


100%|██████████| 25448995/25448995 [04:53<00:00, 86826.77it/s] 


### Parse the multilevel JSON into a CSV file

In [7]:
f = open('data/execution/script_history_fixed.json')
data = json.loads(f.read())

In [8]:
try:
    os.remove('data/execution/check_structure.csv')
except:
    print('File does not exist')
with open('data/execution/check_structure.csv', 'a') as fw:
    #line = 'topProjectName,name,sessionId,executionLogsId,scriptId,scriptName,script_status,label,step_state,step_status,startTime,endTime,duration,lineNum,StepLogsId,lastIssues\n'
    line = 'topProjectName,name,sessionId,executionLogsId,scriptId,scriptName,script_status,label,step_state,step_status,startTime,endTime,duration,lineNum,StepLogsId\n'
    fw.write(line)
    i = 0
    for level1 in tqdm(data):
        # for key, value in level1.items():
        #     #if i == 100: break
        
                     
        try:
            executionLogs_str = json.dumps(level1['executionLogs'])
            executionLogs_json = json.loads(executionLogs_str)
            for level2 in executionLogs_json:
                for key2, value2 in level2.items():
                    #try:
                    startTime_str = str(level2['startTime'])
                    startTime_str = startTime_str[0:10]
                    startTime = datetime.strptime(startTime_str, '%Y-%m-%d')
                    oldestHistory = datetime.strptime(extract_date, '%Y-%m-%d')
                    executionStepLogs_str = json.dumps(level2['executionStepLogs'])
                    executionStepLogs_json = json.loads(executionStepLogs_str)
                    if key2 == 'executionStepLogs' and startTime >= oldestHistory:

                        topProjectName = str(level1['topProjectName'])
                        name_str = str(level1['name'])
                        # Remove any commas from name 
                        check = (',' in name_str)
                        if check == True:
                            name_str = re.sub(',', ';', name_str)                            
                        sessionId_str = str(level2['sessionId'])
                        id_str = str(level2['id'])
                        scriptId_str = str(level2['scriptId'])
                        scriptName_str = str(level2['scriptName'])
                        # Remove any commas from scriptName 
                        check = (',' in scriptName_str)
                        if check == True:
                            scriptName_str = re.sub(',', ';', scriptName_str)                             
                        state_str = str(level2['state'])
                        status2_str = str(level2['status'])
                        # lastIssues_str = str(level2['lastIssues'])
                        # Remove any commas from lastIssues_str
                        # check = (',' in lastIssues_str)
                        # if check == True:
                        #     lastIssues_str = re.sub(',', '~', lastIssues_str)
                        # check = ('\n' in lastIssues_str)
                        # if check == True:
                        #     lastIssues_str = re.sub('\n', '|', lastIssues_str)                         


                        for level3 in executionStepLogs_json:
                            
                            #for key3, value3 in level3.items():
                            status3_str = str(level3['status'])
                            label_str = str(level3['label'])
                            
                            # These lines replace long text test step labels with short text (that does not wrap)
                            check = ('Click Dynamics 365' in label_str)
                            if check == True:
                                label_str = 'Click Dynamics 365'
                            check = ('Click Sales Hub' in label_str)
                            if check == True:
                                label_str = 'Click Sales Hub'
                            check = ('Type value function eaExecuteVariableValue()' in label_str)
                            if check == True:
                                label_str = 'Type value function eaExecuteVariableValue()'
                            check = ('Select from function eaExecuteVariableValue()' in label_str)
                            if check == True:
                                label_str = 'Select from function eaExecuteVariableValue()'                                    
                            check = ('Select lookup values function eaExecuteVariableValue()' in label_str)
                            if check == True:
                                label_str = 'Select lookup values function eaExecuteVariableValue()'
                            # Remove any commas from label_str 
                            check = (',' in label_str)
                            if check == True:
                                label_str = re.sub(',', ';', label_str)
                            # Now replace all newlines that remain in label_str
                            check = ('\n' in label_str)
                            if check == True:
                                label_str = re.sub('\n', '/nl/', label_str)
                            # Done with replacing text

                            startTime_str = str(level3['startTime'])
                            endTime_str = str(level3['endTime'])
                            duration_str = str(level3['duration'])
                            lineNum_str = str(level3['lineNum'])
                            StepLogs_id_str = str(level3['id'])

                            i = i + 1

                            #line = topProjectName+','+name_str+','+sessionId_str+','+id_str+','+scriptId_str+','+scriptName_str+','+status2_str+','+label_str+','+state_str+','+status3_str+','+startTime_str+','+endTime_str+','+duration_str+','+lineNum_str+','+StepLogs_id_str+','+lastIssues_str+'\n'
                            line = topProjectName+','+name_str+','+sessionId_str+','+id_str+','+scriptId_str+','+scriptName_str+','+status2_str+','+label_str+','+state_str+','+status3_str+','+startTime_str+','+endTime_str+','+duration_str+','+lineNum_str+','+StepLogs_id_str+'\n'
                            fw.write(line)
                            if level3['id'] == 2:
                                print(line)                                
                        # except:
                        #     print('Exception on Level2 for Level1 item '+str(i))
        except:
            #print('Exception for Level1 item '+str(i))
            pass       

    # Previous runtime = 2min 40s   

100%|██████████| 963/963 [02:51<00:00,  5.60it/s]


### Read the denormalized CSV file into a Pandas dataframe

In [9]:
df = pd.read_csv('data/execution/check_structure.csv')

# Previous runtime = 2.2s

In [10]:
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df.rename(columns={'label':'StepLabel'}, inplace=True)
df.rename(columns={'name':'testName'}, inplace=True)

In [11]:
# df['lastIssues'] = df['lastIssues'].str.replace('|','\n', regex=False)
# df['lastIssues'] = df['lastIssues'].str.replace('~',',', regex=False)

In [12]:
df = df.dropna(subset = ['testName']).copy()

In [13]:
mask = (df['startTime'] == 'None') 
df['startTime'].mask(mask,'2022-01-01T00:00:00.000Z',inplace=True)

In [14]:
mask = (df['endTime'] == 'None') 
df['endTime'].mask(mask,'2022-01-01T00:00:00.000Z',inplace=True)

In [15]:
mask = (df['duration'] == 'None') 
df['duration'].mask(mask,'0',inplace=True)
df['duration'] = pd.to_numeric(df['duration'])

In [16]:
df['duration_sec'] = df.duration/1000
df['duration_mins'] = df.duration/1000/60

In [17]:
df['date'] = pd.to_datetime(df['startTime'], format='%Y-%m-%d').dt.date
df['date'] = pd.to_datetime(df['date'])


In [18]:
df['time'] = pd.to_datetime(df['startTime']).dt.time

In [19]:
df['time_dec'] = df['time'].astype(str)
df['time_dec'] = df['time_dec'].str[:5]
df['time_dec'] = df['time_dec'].str.replace(':','.')
df['time_dec'] = df['time_dec'].astype(float)

### Add a column to indicate Runs of each test script
As there is no other field available via the API, this field is generated each time the lineNum == 1.  This assumes that there will always be a step 1 in every test case.
Before this can be done, first sort the whole dataframe

In [20]:
df.sort_values(by=['executionLogsId','StepLogsId'], inplace=True)
df = df.reset_index(drop=True)

In [21]:
# This is old code to allocate a 'run' number and increment it every time the lineNum == 1

# df['run'] = 0
# run = 0
# for idx in df.index:
#     if df['lineNum'][idx] == 1:
#         run = run + 1
#     df['run'][idx] = run

 # Previous runtime = 31.4s    

In [22]:
# This is the new code to allocate a 'run' number and increment it every time the lineNum == 1.  This was generated by ChatGPT based on the code above :-)

df['run'] = (df['lineNum'] == 1).cumsum()

### Generate a field to indicate where test scripts have run to the last step of the script
This assumes that the last step is "End script"
First sort the dataframe by Runs
Then add a new text field that is a concatenation of all the step labels for each run
Finally, check which of those strings contain the text "End script"

In [23]:
runs_np = df.run.unique()
end_script_dict = {}
for i in runs_np:
    end_script_dict[i] = 'Script stopped'
step_error_dict = {}
for i in runs_np:
    step_error_dict[i] = 'No step errors'

In [24]:
df.sort_values(by=['run'], inplace=True)
df = df.reset_index(drop=True)

In [25]:
for idx in df.index:
    #if run == 10: break
    run = df['run'][idx]
    if df['StepLabel'][idx] == 'End script':
        end_script_dict[run] = 'Script completed'
    if df['step_status'][idx] == 'ERROR':
        step_error_dict[run] = 'One or more step errors'

In [26]:
end_script_df = pd.DataFrame.from_dict(end_script_dict,orient ='index',columns=['script_completion'])
end_script_df.reset_index(inplace=True)
end_script_df.rename(columns={'index':'run'}, inplace=True)

In [27]:
step_error_df = pd.DataFrame.from_dict(step_error_dict,orient ='index',columns=['step_error'])
step_error_df.reset_index(inplace=True)
step_error_df.rename(columns={'index':'run'}, inplace=True)

In [28]:
df = pd.merge(
    df,
    end_script_df,
    how="inner",
    on=None,
    left_on='run',
    right_on='run',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

In [29]:
df = pd.merge(
    df,
    step_error_df,
    how="inner",
    on=None,
    left_on='run',
    right_on='run',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

In [30]:
df.rename(columns={'script_status': 'EA_script_status','step_state': 'EA_step_state','step_status': 'EA_step_status','step_error': 'IPT_step_error','script_completion': 'IPT_script_completion'}, inplace=True)

In [31]:
df['KPI'] = 'Not mapped'

In [32]:
# K1
mask = (df['scriptName'].str.contains('Place a sales local order',case=False))
df['KPI'].mask(mask,'K01 Place a sales local order',inplace=True)

# K2
mask = (df['scriptName'].str.contains('Place a B2B local order',case=False))
df['KPI'].mask(mask,'K02 Place a B2B/EDI order and Send Confirmation',inplace=True)

# K3 Place Service Portal Order
# No test script

# K4
mask = (df['scriptName'].str.contains('Amend an existing sales order',case=False))
df['KPI'].mask(mask,'K04 Amend an existing sales order',inplace=True)

# K5
mask = (df['scriptName'].str.contains('Trade Returns Order',case=False))
df['KPI'].mask(mask,'K05 Place a Trade Returns Order',inplace=True)

# K6
mask = (df['scriptName'].str.contains('Submit a B2B Remittance',case=False))
df['KPI'].mask(mask,'K06 B2B Remittance',inplace=True)

# K7 B2B Remittance Adjustment
# No test script

#K8
mask = (df['scriptName'].str.contains('Receipted payment processing',case=False))
df['KPI'].mask(mask,'K08 Receipted payment processing SA Only at the moment',inplace=True)

# K9 aDSD Batch Job posting invoice for HHD billing document 
# No test script

# K10
mask = (df['scriptName'].str.contains('Credit status check',case=False))
df['KPI'].mask(mask,'K10 Credit status check',inplace=True)

# K11
mask = (df['scriptName'].str.contains('Send Load to Roadnet',case=False))
df['KPI'].mask(mask,'K11 Send Load to Roadnet load for Planning',inplace=True)

# K12
mask = (df['scriptName'].str.contains('Receive load from Roadnet',case=False))
df['KPI'].mask(mask,'K12 Receive load from Roadnet into D365',inplace=True)

# K13b
mask = (df['scriptName'].str.contains('Release to warehouse',case=False) & df['scriptName'].str.contains('Roadnet loads',case=False))
df['KPI'].mask(mask,'K13b Release to Warehouse & Complete Picking Work (Roadnet loads)',inplace=True)
# Release to Warehouse (Roadnet loads) & Complete Picking Work & Process OOS

# K13
mask = (df['scriptName'].str.contains('Release to Warehouse',case=False) & df['scriptName'].str.contains('manual',case=False))
df['KPI'].mask(mask,'K13 Release to Warehouse (manually planned loads)',inplace=True)

# K14 Complete Picking Work & Process OOS 
# Not mapped??


# K14b
mask = (df['scriptName'].str.contains('Complete Picking Work',case=False) & df['scriptName'].str.contains('manual loads',case=False))
df['KPI'].mask(mask,'K14b Complete Picking Work & Process OOS (manually planned loads)',inplace=True)

# K15
mask = (df['scriptName'].str.contains('aDSD Load confirmation',case=False))
df['KPI'].mask(mask,'K15 aDSD Load confirmation',inplace=True)

# K16
mask = (df['scriptName'].str.contains('Load upload',case=False) & df['scriptName'].str.contains('Settlement',case=False))
df['KPI'].mask(mask,'K16 Load upload & Settlement',inplace=True)

# K17
mask = (df['scriptName'].str.contains('Create a cost estimate',case=False))
df['KPI'].mask(mask,'K17 Create a cost estimate for all standard costed procured materials',inplace=True)

# K18
mask = (df['scriptName'].str.contains('Imported Statistical',case=False))
df['KPI'].mask(mask,'K18 Import Actual Statistical entries, into Cost Accounting',inplace=True)

# K19
mask = (df['scriptName'].str.contains('Distribute the range of items pending prices',case=False))
df['KPI'].mask(mask,'K19 Distribute the range of items pending prices to another site',inplace=True)

# K20
mask = ((df['scriptName'].str.contains('Cost Rollup',case=False) & df['scriptName'].str.contains('Cost Allocation',case=False)) | df['scriptName'].str.contains('Maintain cost distribution table',case=False))
df['KPI'].mask(mask,'K20 Run Cost Rollup and Cost Allocation Policies',inplace=True)

# K21
mask = (df['scriptName'].str.contains('Place SFA order',case=False))
df['KPI'].mask(mask,'K21 Place SFA order and Send Confirmation',inplace=True)

In [33]:
df.sort_values(by=['KPI', 'scriptId', 'scriptName', 'sessionId', 'run', 'lineNum'], inplace=True)
df = df.reset_index(drop=True)

In [34]:
df.drop(columns={'topProjectName'}, inplace=True, axis=1)

In [35]:
df = df.reindex(columns=['KPI', 'scriptId', 'scriptName', 'sessionId', 'run', 'EA_script_status', 'IPT_script_completion', 'IPT_step_error', 'lineNum', 'StepLabel', 'EA_step_state', 'EA_step_status', 'startTime', 'endTime', 'duration', 'duration_sec', 'duration_mins', 'date', 'time_dec'])

In [36]:
raise SystemExit("Initial extraction completed")

SystemExit: Initial extraction completed

/home/wo/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Filter the dataset for output on a specific test cycle

In [37]:
rundate = '2023-04-05'

In [38]:
exclude_scripts = ['TEST TIMINGS','Test - 10 sessions', 'Gary Test']

In [61]:
#df1 = df.query("((date == '2023-02-02' and time_dec > 13.0) or (date == '2023-02-02' and time_dec > 0.0)) and scriptName not in @exclude_scripts").copy()  # Remember time is UTC
#df1 = df.query("date == '2023-02-05' and scriptName not in @exclude_scripts").copy()  # Remember time is UTC
df1 = df.query("date == @rundate and time_dec > 10.0 and scriptName not in @exclude_scripts").copy()  # Remember time is UTC

In [62]:
df1 = df1.reset_index(drop=True)

In [63]:
# New code to keep lastIssues only for those lineNums with a corresponding number - generated by ChatGPT :-)

# df1['lastIssues'].fillna(value='', inplace=True)
# mask = df1.apply(lambda x: '[ERROR] Step {}.'.format(str(int(x['lineNum']))) in x['lastIssues'], axis=1)
# df1.loc[~mask, 'lastIssues'] = ''

In [64]:
# Old code to keep lastIssues only for those lineNums with a corresponding number

""" for i in tqdm(range(len(df1))):
    ln = str(df1.loc[i, 'lineNum'])
    lastIssue_str = str(df1.loc[i, 'lastIssues'])
    string1 = '[ERROR] Step '+ln+'.'
    check = (string1 in lastIssue_str)
    if check != True:
        df1.loc[i, 'lastIssues'] = '' """

# Previous runtime = 30.4s 

" for i in tqdm(range(len(df1))):\n    ln = str(df1.loc[i, 'lineNum'])\n    lastIssue_str = str(df1.loc[i, 'lastIssues'])\n    string1 = '[ERROR] Step '+ln+'.'\n    check = (string1 in lastIssue_str)\n    if check != True:\n        df1.loc[i, 'lastIssues'] = '' "

In [65]:
successful_runs = df1[df1['IPT_script_completion'] == 'Script completed'].groupby('KPI').agg({'run': 'nunique'}).reset_index()


In [66]:
incomplete_runs = df1[df1['IPT_script_completion'] != 'Script completed'].groupby('KPI').agg({'run': 'nunique'}).reset_index()
incomplete_runs.rename(columns={'run': 'Incomplete Runs', 'KPI': 'KPI Process'}, inplace=True)

### Mark those Test Script Steps that must be included in the time measurement

In [67]:
df1['KPI_no'] = ''

In [68]:
df1['Include_in_measure'] = False

In [69]:
kpi_numbers = ['K11', 'K12', 'K13', 'K14', 'K15', 'K16', 'K17', 'K18', 'K19', 'K20']

In [70]:
for idx in tqdm(df1.index):
    label = df1['StepLabel'][idx]
    prefix = label[0:3]
    if prefix in kpi_numbers:
        if label[3] == 'b':
            df1.at[idx, 'KPI_no'] = prefix + 'b'
        else:
            df1.at[idx, 'KPI_no'] = prefix  
        df1.at[idx, 'Include_in_measure'] = True

100%|██████████| 77562/77562 [00:00<00:00, 187304.45it/s]


In [71]:
filename = 'data/execution/' + rundate +  '_results_filtered.csv'
df1.to_csv(filename,index=False)

In [72]:
df2 = df1.query("Include_in_measure == True and EA_step_status != 'WARNING' and EA_step_status != 'ERROR'").copy()

In [73]:
df2 = df2.drop_duplicates(keep='first')
df2 = df2.reset_index(drop=True)

In [74]:
# 90th Percentile
def q90(x):
    return x.quantile(0.9)


kpi_results = df2.groupby(['KPI_no']).agg({'KPI': 'first', 'duration_sec': ['mean', 'max', q90, 'std'], 'run': 'count'}).reset_index()

# Flatten multi-level columns
kpi_results.columns = ['_'.join(col) for col in kpi_results.columns]
kpi_results = kpi_results.reset_index(drop=True)

In [75]:
kpi_results.rename(columns={'KPI_no_': 'KPI_no', 'KPI_first':'KPI Process', 'duration_sec_mean': 'Average', 'duration_sec_max': 'Max', 'duration_sec_q90': '90th_%', 'duration_sec_std': 'SD', 'run_count': 'Successful Runs'}, inplace=True)

In [76]:
kpi_results = pd.merge(
    kpi_results,
    incomplete_runs,
    how="left",
    on=None,
    left_on='KPI Process',
    right_on='KPI Process',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

kpi_results['Incomplete Runs'] = kpi_results['Incomplete Runs'].fillna(0)

In [77]:
kpi_results = kpi_results.round(decimals=1)

In [78]:
kpi_results

,KPI_no,KPI Process,Average,Max,90th_%,SD,Successful Runs,Incomplete Runs
0,K12,K12 Receive load from Roadnet into D365,291.2,425.8,349.9,65.4,8,0.0
1,K13,K13 Release to Warehouse (manually planned loads),32.0,50.4,50.3,12.0,10,1.0
2,K14,K13 Release to Warehouse (manually planned loads),162.3,318.4,223.1,81.4,10,1.0
3,K13b,K13b Release to Warehouse & Complete Picking W...,11.7,44.8,18.6,6.4,127,2.0
4,K14b,K13b Release to Warehouse & Complete Picking W...,9.2,72.0,21.6,10.4,562,2.0
5,K15,K15 aDSD Load confirmation,61.0,203.1,82.9,18.1,725,4.0


In [79]:
filename = 'data/execution/' + rundate +  '_kpi_results.xlsx'
kpi_results.to_excel(filename,index=False)

In [80]:
raise SystemExit("Stop right here!")

SystemExit: Stop right here!

/home/wo/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Experimental from here on

### Aggregate to extract measures

In [ ]:
""" def summary(x):
    runs = x['executionLogsId'].nunique()
    runs = float(runs)
    duration = x['duration'].sum()/1000
    duration_90 = x['duration'].quantile(0.9).sum()/1000
    mean_s = x['duration'].mean()/1000
    mean_90 = x['duration'].quantile(0.9).mean()/1000
    count = duration/mean_s
    result = {
        #'aggr_duration': duration,
        #'aggr_duration_minutes': duration/60,
        #'runs': runs,
        'mean': mean_s,
        'mean_90': mean_90,
        'mean_minutes': duration/runs/60
        #'steps': count
    }
    return pd.Series(result).round(1)

#df2.groupby(['scriptName','sessionId']).apply(summary).to_html('data/execution/result.html')
df2.groupby(['testName', 'executionLogsId']).apply(summary).to_html('data/execution/result.html')
#df2.groupby(['testName', 'executionLogsId']).quantile(.90).apply(summary).to_html('data/execution/result.html')
#df2.groupby(['testName', 'executionLogsId']).apply(summary).to_excel('data/execution/result_groupby.xlsx')
#df2.groupby(['testName', 'executionLogsId']).agg({'executionLogsId': {'nunique'}, 'duration' : {'sum'}})
#df2.groupby(['testName']).apply(summary).to_html('data/execution/result.html') """

### From here on the script is disabled.

In [ ]:
# Insert this to prevent the cells below from trying to write to the sql table
raise SystemExit("Stop right here!")

### Get ready to export to SQL

In [ ]:
mask = (df['date'] > '2023-01-10')
#df.loc[(df['date'] > '2023-01-10')]
df = df[mask]

In [ ]:
# https://learn.microsoft.com/en-us/sql/machine-learning/data-exploration/python-dataframe-pandas?view=sql-server-ver16
# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
server = 'tcp:csazniptlsql01.database.windows.net,1433' 
database = 'CSAZN-INT-IPT-L-DSQ-01' 
username = 'ccbaadmin' 
password = '@FireBase123' 
# ENCRYPT defaults to yes starting in ODBC Driver 18. It's good to always specify ENCRYPT=yes on the client side to avoid MITM attacks.
cnxn = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server+';DATABASE='+database+';ENCRYPT=yes;UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [ ]:
# cursor = cnxn.cursor()

# cursor.execute('''
# 		CREATE TABLE ea_script_history (
# 			records_no int,
#             topProjectName nvarchar(255),
#             testName nvarchar(255),
#             sessionId nvarchar(255),
#             id int,
#             scriptId int,
#             scriptName nvarchar(255),
#             state nvarchar(255),
#             status nvarchar(255),
#             StepLabel nvarchar(255),
#             startTime datetime,
#             endTime datetime,
#             duration int,
#             lineNum int,
#             StepLogs_id int
# 			)
#                ''')

# cnxn.commit()

In [ ]:
# cnxn = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server+';DATABASE='+database+';ENCRYPT=yes;UID='+username+';PWD='+ password)
# cursor = cnxn.cursor()
# # Insert Dataframe into SQL Server:
# i = 0
# for index, row in df.iterrows():
#      cursor.execute("INSERT INTO ea_script_history (records_no,topProjectName,testName,sessionId,id,scriptId,scriptName,state,status,StepLabel,startTime,endTime,duration,lineNum,StepLogs_id) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", row.records_no, row.topProjectName, row.testName, row.sessionId, row.id, row.scriptId, row.scriptName, row.state, row.status, row.StepLabel, row.startTime, row.endTime, row.duration, row.lineNum, row.StepLogs_id)
#      i = i + 1
#      if i == 1000:
#           i = 0
#           cnxn.commit()          
# cnxn.commit()
# cursor.close()

In [ ]:
# cnxn = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server+';DATABASE='+database+';ENCRYPT=yes;UID='+username+';PWD='+ password)
# cursor = cnxn.cursor()
# # Insert Dataframe into SQL Server:
# for index, row in df.iterrows():
#      cursor.execute("INSERT INTO ea_script_history (startTime) values(?)", row.startTime)
# cnxn.commit()
# cursor.close()

In [ ]:
# Delete all the rows in the SQl script history table
# cursor = cnxn.cursor()
# cursor.execute("DELETE FROM ea_script_history")
# cnxn.commit()
# cursor.close()

In [ ]:
""" # Drop the SQl script history table (it must be re-created)
cursor = cnxn.cursor()
cursor.execute("DROP Table ea_script_history")
#cursor.execute("DROP Table temp_table")
cnxn.commit()
cursor.close() """

In [ ]:
try_steplabel = 'K14b Click Complete work'

In [ ]:
try_kpi = 'K14 '

In [ ]:
if try_kpi in try_steplabel:
    print('Match')